<a href="https://colab.research.google.com/github/arpitmalhotra009/Data_Warehousing/blob/main/Data_Warehousing_using_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datawarehousing with MySQL - StartUP

based on the codes available with the book Dimensional Datawarehousing with MySQL https://brainysoftware.com/9780975212820 [archive](https://drive.google.com/open?id=1BeIC5uEMGu85PADHRN-Zpn3lmZ3ihrkE)

# Install MySQL

In [ ]:
!apt -y install mysql-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libencode-locale-perl libevent-core-2.1-6
  libfcgi-perl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl
  libhttp-date-perl libhttp-message-perl libio-html-perl
  liblwp-mediatypes-perl libtimedate-perl liburi-perl mysql-client-5.7
  mysql-client-core-5.7 mysql-server-5.7 mysql-server-core-5.7
Suggested packages:
  libdata-dump-perl libipc-sharedcache-perl libwww-perl mailx tinyca
The following NEW packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libencode-locale-perl libevent-core-2.1-6
  libfcgi-perl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl
  libhttp-date-perl libhttp-message-perl libio-html-perl
  liblwp-mediatypes-perl libtimedate-perl liburi-perl mysql-client-5.7
  mysql-client-core-5.7 mysql-server mysql-server-5.7 mysql-server-core-5.7
0 upgraded, 19 

In [ ]:
!/etc/init.d/mysql restart

 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
No directory, logging in with HOME=/
   ...done.


In [ ]:
!mysql -e 'show databases'

+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| sys                |
+--------------------+


#Get Code & Data Files

In [ ]:
!wget https://brainysoftware.com/source/9780975212820.zip

--2020-10-17 16:41:48--  https://brainysoftware.com/source/9780975212820.zip
Resolving brainysoftware.com (brainysoftware.com)... 192.241.166.210
Connecting to brainysoftware.com (brainysoftware.com)|192.241.166.210|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 166447 (163K) [application/zip]
Saving to: ‘9780975212820.zip’

9780975212820.zip   100%[===================>] 162.55K  --.-KB/s    in 0.1s    

2020-10-17 16:41:49 (1.27 MB/s) - ‘9780975212820.zip’ saved [166447/166447]



In [ ]:
!unzip 9780975212820.zip

Archive:  9780975212820.zip
   creating: scripts/
  inflating: scripts/across_all_dimensions.sql  
  inflating: scripts/across_date.sql  
  inflating: scripts/across_date_customer.sql  
  inflating: scripts/across_date_product.sql  
  inflating: scripts/additive_data.sql  
  inflating: scripts/add_campaign_session.sql  
  inflating: scripts/add_four_milestones.sql  
  inflating: scripts/add_sales_order_quantity.sql  
  inflating: scripts/add_two_sales_orders.sql  
  inflating: scripts/annual_aggregation.sql  
  inflating: scripts/annual_tables.sql  
  inflating: scripts/balance_across_months.sql  
  inflating: scripts/balance_across_products.sql  
  inflating: scripts/band_dim.sql    
  inflating: scripts/band_ini.sql    
  inflating: scripts/band_regular.sql  
  inflating: scripts/campaign_session.sql  
  inflating: scripts/campaign_session_20.sql  
  inflating: scripts/campaign_session_21.sql  
  inflating: scripts/campaign_session_path.sql  
  inflating: scripts/campaign_stg_20.sql 

#Part 1 - Fundamentals

## Chapter 1 Initial Setup

In [ ]:
#Create User ID
#!cat scripts/create_user_id.sql
!mysql < scripts/create_user_id.sql

### Creating the Databases
There are two databases you need to create, source and dw. The source database stores your data, namely the data that will be the source of your data warehouse. The dw database is for the data warehouse.



In [ ]:
#!cat scripts/create_databases.sql
!mysql -udwid -ppw < scripts/create_databases.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "show databases like 'dw'"

+---------------+
| Database (dw) |
+---------------+
| dw            |
+---------------+


In [ ]:
!mysql dw -e "show databases like 'source';"

+-------------------+
| Database (source) |
+-------------------+
| source            |
+-------------------+


### Creating Data Warehouse Tables
The next step is to create data warehouse tables in the dw database. You can use the create_dw_tables.sql script


In [ ]:
#!cat scripts/create_dw_tables.sql
!mysql -udwid -ppw < scripts/create_dw_tables.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "show tables"

+------------------+
| Tables_in_dw     |
+------------------+
| customer_dim     |
| date_dim         |
| order_dim        |
| product_dim      |
| sales_order_fact |
+------------------+


### Generating Surrogate Keys
Generate surrogate keys using the customer_sk.sql script 


In [ ]:
!mysql -udwid -ppw < scripts/customer_sk.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


### Populating the customer_dim dimension table with data

In [ ]:
!mysql -udwid -ppw < scripts/more_customer_sk.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from customer_dim"

+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name    | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Big Customers    | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2020-10-17     | 9999-12-31  |
|           2 |               2 | Small Stores     | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2020-10-17     | 9999-12-31  |
|           3 |               3 | Medium Retailers | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2020-10-17     | 9999-12-31  |
|           4 |       

CURRENT_DATE function used in the customer_sk.sql script uses the operating system date to populate the effective_date column. Therefore we have to update MySQL date by updating the date to the specified date. 


In [ ]:
#since it was not possible to set the system date on Colab, the date was changed manually
!mysql dw -e "update customer_dim set effective_date = '2007-02-01' "

In [ ]:
!mysql dw -e "select * from customer_dim"

+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name    | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Big Customers    | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2007-02-01     | 9999-12-31  |
|           2 |               2 | Small Stores     | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2007-02-01     | 9999-12-31  |
|           3 |               3 | Medium Retailers | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2007-02-01     | 9999-12-31  |
|           4 |       

In [ ]:
!mysql dw -e "CREATE TABLE customer_stg \
(customer_number INT, customer_name CHAR (30), customer_street_address CHAR (30), customer_zip_code INT (5), customer_city CHAR (30), customer_state CHAR (2))"

In [ ]:
!mysql dw -e "show tables"

+------------------+
| Tables_in_dw     |
+------------------+
| customer_dim     |
| customer_stg     |
| date_dim         |
| order_dim        |
| product_dim      |
| sales_order_fact |
+------------------+


In [ ]:
!mysql dw -e "TRUNCATE customer_stg"

In [ ]:
!mysql dw -e 'LOAD DATA LOCAL INFILE "source_data/customer2.csv " \
INTO TABLE customer_stg \
FIELDS TERMINATED BY "," \
OPTIONALLY ENCLOSED BY "" \
LINES TERMINATED BY "\r\n" \
IGNORE 1 LINES \
( customer_number \
, customer_name \
, customer_street_address \
, customer_zip_code \
, customer_city \
, customer_state )'

In [ ]:
!mysql dw -e  ' desc customer_stg'

+-------------------------+----------+------+-----+---------+-------+
| Field                   | Type     | Null | Key | Default | Extra |
+-------------------------+----------+------+-----+---------+-------+
| customer_number         | int(11)  | YES  |     | NULL    |       |
| customer_name           | char(30) | YES  |     | NULL    |       |
| customer_street_address | char(30) | YES  |     | NULL    |       |
| customer_zip_code       | int(5)   | YES  |     | NULL    |       |
| customer_city           | char(30) | YES  |     | NULL    |       |
| customer_state          | char(2)  | YES  |     | NULL    |       |
+-------------------------+----------+------+-----+---------+-------+


## Chapter 2: Dimension History

The data values stored in a dimension table are called dimensions. The product_dim table in Chapter 1, for example, contains product dimensions.

Most dimensions change over time. Customers change addresses, products get renamed and recategorized, sales orders get corrected, etc. When a dimension changes, such as when a product gets a new category, we must maintain the dimension history. 

Slowly Changing Dimension (SCD) is the technique for implementing dimension history in a dimensional data warehouse

###Slowly Changing Dimension Techniques

There are three variants of the SCD technique: SCD Type 1 (SCD1), SCD Type 2 (SCD2), and SCD Type 3 (SCD3). 

SCD1 updates dimension records by overwriting the existing data-no history of the records is maintained. SCD1 is normally used to directly rectify incorrect data.

SCD2 maintains dimension history by creating newer ‘versions’ of a dimension record whenever its source changes. SCD2 does not delete or modify existing data. 

SCD3 keeps one version of a dimension record. It keeps history by allocating more than one column for a data unit to maintain its history. 


In [ ]:
!mysql dw -e "select * from customer_stg"

+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+
| customer_number | customer_name          | customer_street_address | customer_zip_code | customer_city | customer_state |
+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+
|               1 | Really Large Customers | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             |
|               2 | Small Stores           | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             |
|               3 | Medium Retailers       | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             |
|               4 | Good Companies         | 9500 Scott St.          |             17050 | Mechanicsburg | PA             |
|               5 | Wonderful Shops        | 3333 Rossmoyne Rd.      |             17050 | Mechanicsburg | PA             |
|       

In [ ]:
!mysql dw -e 'select * from customer_dim'

+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name    | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Big Customers    | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2007-02-01     | 9999-12-31  |
|           2 |               2 | Small Stores     | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2007-02-01     | 9999-12-31  |
|           3 |               3 | Medium Retailers | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2007-02-01     | 9999-12-31  |
|           4 |       

In [ ]:
#updating the name of existing customer
!mysql dw -e 'UPDATE customer_dim a, customer_stg b SET a.customer_name = b.customer_name \
WHERE a.customer_number = b.customer_number \
AND a.expiry_date = "9999-12-31" \
AND a.customer_name <> b.customer_name'

In [ ]:
!mysql dw -e 'select * from customer_dim'

+-------------+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name          | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Really Large Customers | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2007-02-01     | 9999-12-31  |
|           2 |               2 | Small Stores           | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2007-02-01     | 9999-12-31  |
|           3 |               3 | Medium Retailers       | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2007-02-01     | 

**It can be seen that the name has been updated in the Dimension Table without saving the History or record of previous name of Customer**

### SCD2

In [ ]:
 !mysql dw -e 'CREATE TABLE product_stg (product_code INT,product_name CHAR(30),product_category CHAR(30))'

In [ ]:
!mysql dw -e 'show tables'

+------------------+
| Tables_in_dw     |
+------------------+
| customer_dim     |
| customer_stg     |
| date_dim         |
| order_dim        |
| product_dim      |
| product_stg      |
| sales_order_fact |
+------------------+


In [ ]:
!mysql dw -e "TRUNCATE product_stg"

In [ ]:
!mysql dw -e 'LOAD DATA LOCAL INFILE "source_data/product2-1.txt " \
INTO TABLE product_stg \
FIELDS TERMINATED BY "" \
OPTIONALLY ENCLOSED BY "" \
LINES TERMINATED BY "\r\n" \
IGNORE 1 LINES \
( product_code \
, product_name \
, product_category )'

In [ ]:
 # Viewing data in product staging table
 !mysql dw -e 'select * from product_stg'

+--------------+--------------+------------------+
| product_code | product_name | product_category |
+--------------+--------------+------------------+
|            1 | Hard Disk    | Storage          |
|            2 | Floppy Drive | Storage          |
+--------------+--------------+------------------+


In [ ]:
 # Description of product dim table
 !mysql dw -e 'desc product_dim'

+------------------+----------+------+-----+---------+----------------+
| Field            | Type     | Null | Key | Default | Extra          |
+------------------+----------+------+-----+---------+----------------+
| product_sk       | int(11)  | NO   | PRI | NULL    | auto_increment |
| product_code     | int(11)  | YES  |     | NULL    |                |
| product_name     | char(30) | YES  |     | NULL    |                |
| product_category | char(30) | YES  |     | NULL    |                |
| effective_date   | date     | YES  |     | NULL    |                |
| expiry_date      | date     | YES  |     | NULL    |                |
+------------------+----------+------+-----+---------+----------------+


In [ ]:
 #Checking for data in product_dim table 
 !mysql dw -e 'Select * from product_dim'

In [ ]:
# Adding data in product_dim table
!mysql -udwid -ppw <scripts/scd2.sql 

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e 'select * from product_dim'

+------------+--------------+--------------+------------------+----------------+-------------+
| product_sk | product_code | product_name | product_category | effective_date | expiry_date |
+------------+--------------+--------------+------------------+----------------+-------------+
|          1 |            1 | Hard Disk    | Storage          | 2020-10-17     | 9999-12-31  |
|          2 |            2 | Floppy Drive | Storage          | 2020-10-17     | 9999-12-31  |
+------------+--------------+--------------+------------------+----------------+-------------+


In [ ]:
# updating date since it cant be updated automatically  in product_dim
!mysql dw -e 'update product_dim set effective_date = "2007-02-03" '

In [ ]:
!mysql dw -e 'select * from product_dim'

+------------+--------------+--------------+------------------+----------------+-------------+
| product_sk | product_code | product_name | product_category | effective_date | expiry_date |
+------------+--------------+--------------+------------------+----------------+-------------+
|          1 |            1 | Hard Disk    | Storage          | 2007-02-03     | 9999-12-31  |
|          2 |            2 | Floppy Drive | Storage          | 2007-02-03     | 9999-12-31  |
+------------+--------------+--------------+------------------+----------------+-------------+


In [ ]:
!mysql dw -e "TRUNCATE product_stg"

In [ ]:
!mysql dw -e "select * from product_stg"

It can be seen that Now no data is present in product staging table

In [ ]:
!mysql dw -e 'LOAD DATA LOCAL INFILE "source_data/product2-2.txt " \
INTO TABLE product_stg \
FIELDS TERMINATED BY "" \
OPTIONALLY ENCLOSED BY "" \
LINES TERMINATED BY "\r\n" \
IGNORE 1 LINES \
( product_code \
, product_name \
, product_category )'

In [ ]:
 # Viewing data in product staging table
 !mysql dw -e 'select * from product_stg'

+--------------+-----------------+------------------+
| product_code | product_name    | product_category |
+--------------+-----------------+------------------+
|            1 | Hard Disk Drive | Storage          |
|            2 | Floppy Drive    | Storage          |
|            3 | LCD Panel       | Monitor          |
+--------------+-----------------+------------------+


In [ ]:
!mysql -udwid -ppw <scripts/scd2.sql 

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e 'select * from product_dim'

+------------+--------------+-----------------+------------------+----------------+-------------+
| product_sk | product_code | product_name    | product_category | effective_date | expiry_date |
+------------+--------------+-----------------+------------------+----------------+-------------+
|          1 |            1 | Hard Disk       | Storage          | 2007-02-03     | 2020-10-16  |
|          2 |            2 | Floppy Drive    | Storage          | 2007-02-03     | 9999-12-31  |
|          4 |            1 | Hard Disk Drive | Storage          | 2020-10-17     | 9999-12-31  |
|          5 |            3 | LCD Panel       | Monitor          | 2020-10-17     | 9999-12-31  |
+------------+--------------+-----------------+------------------+----------------+-------------+


In [ ]:
!mysql dw -e 'update product_dim set expiry_date ="2007-02-04" where product_name ="Hard Disk" '

In [ ]:
!mysql dw -e 'select * from product_dim'

+------------+--------------+-----------------+------------------+----------------+-------------+
| product_sk | product_code | product_name    | product_category | effective_date | expiry_date |
+------------+--------------+-----------------+------------------+----------------+-------------+
|          1 |            1 | Hard Disk       | Storage          | 2007-02-03     | 2007-02-04  |
|          2 |            2 | Floppy Drive    | Storage          | 2007-02-03     | 9999-12-31  |
|          4 |            1 | Hard Disk Drive | Storage          | 2020-10-17     | 9999-12-31  |
|          5 |            3 | LCD Panel       | Monitor          | 2020-10-17     | 9999-12-31  |
+------------+--------------+-----------------+------------------+----------------+-------------+


In [ ]:
!mysql dw -e 'update product_dim set product_sk =3, effective_date ="2007-02-05" where product_name ="Hard Disk Drive" '

In [ ]:
!mysql dw -e 'select * from product_dim'

+------------+--------------+-----------------+------------------+----------------+-------------+
| product_sk | product_code | product_name    | product_category | effective_date | expiry_date |
+------------+--------------+-----------------+------------------+----------------+-------------+
|          1 |            1 | Hard Disk       | Storage          | 2007-02-03     | 2007-02-04  |
|          2 |            2 | Floppy Drive    | Storage          | 2007-02-03     | 9999-12-31  |
|          3 |            1 | Hard Disk Drive | Storage          | 2007-02-05     | 9999-12-31  |
|          5 |            3 | LCD Panel       | Monitor          | 2020-10-17     | 9999-12-31  |
+------------+--------------+-----------------+------------------+----------------+-------------+


In [ ]:
!mysql dw -e 'update product_dim set product_sk =4, effective_date ="2007-02-05" where product_name ="LCD Panel" '

In [ ]:
!mysql dw -e 'select * from product_dim'

+------------+--------------+-----------------+------------------+----------------+-------------+
| product_sk | product_code | product_name    | product_category | effective_date | expiry_date |
+------------+--------------+-----------------+------------------+----------------+-------------+
|          1 |            1 | Hard Disk       | Storage          | 2007-02-03     | 2007-02-04  |
|          2 |            2 | Floppy Drive    | Storage          | 2007-02-03     | 9999-12-31  |
|          3 |            1 | Hard Disk Drive | Storage          | 2007-02-05     | 9999-12-31  |
|          4 |            3 | LCD Panel       | Monitor          | 2007-02-05     | 9999-12-31  |
+------------+--------------+-----------------+------------------+----------------+-------------+


## Chapter 3: Measure Additivity
A measure always has numeric values. One of the most important characteristics of a fact table pertaining to measures is measure additivity. With regard to additivity, a measure may be fully-additive or semi-additive. If you can sum the values of a measure in all situations, the measure is fully-additive. If you can only add up its values in some situations, it is semi-additive. Understanding measure additivity is key to selecting a data item as a measure in a fact table. The order_amount measure in the sales_order_fact table, for example, is fully-additive because you can correctly add up this measure across any one and all of its dimensions. This means, you can correctly sum the order_amount measure in your sales_order_fact table any time, for any product, any customer, and any order.

**Fully-Additive Measures**

A measure is fully additive if the total of its individual values across any one dimension is the same as the total across any other dimension and across any combination of some or all dimensions


In [ ]:
!mysql -udwid -ppw <scripts/additive_data.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e 'select * from order_dim'

+----------+--------------+----------------+-------------+
| order_sk | order_number | effective_date | expiry_date |
+----------+--------------+----------------+-------------+
|        1 |            1 | 2020-10-17     | 9999-12-31  |
|        2 |            2 | 2020-10-17     | 9999-12-31  |
|        3 |            3 | 2020-10-17     | 9999-12-31  |
|        4 |            4 | 2020-10-17     | 9999-12-31  |
|        5 |            5 | 2020-10-17     | 9999-12-31  |
|        6 |            6 | 2020-10-17     | 9999-12-31  |
|        7 |            7 | 2020-10-17     | 9999-12-31  |
|        8 |            8 | 2020-10-17     | 9999-12-31  |
|        9 |            9 | 2020-10-17     | 9999-12-31  |
|       10 |           10 | 2020-10-17     | 9999-12-31  |
+----------+--------------+----------------+-------------+


In [ ]:
!mysql dw -e 'update order_dim set effective_date ="2007-02-05" where order_sk in (1,2,3,4,5,6,7,8,9,10) '

In [ ]:
!mysql dw -e 'select * from order_dim'

+----------+--------------+----------------+-------------+
| order_sk | order_number | effective_date | expiry_date |
+----------+--------------+----------------+-------------+
|        1 |            1 | 2007-02-05     | 9999-12-31  |
|        2 |            2 | 2007-02-05     | 9999-12-31  |
|        3 |            3 | 2007-02-05     | 9999-12-31  |
|        4 |            4 | 2007-02-05     | 9999-12-31  |
|        5 |            5 | 2007-02-05     | 9999-12-31  |
|        6 |            6 | 2007-02-05     | 9999-12-31  |
|        7 |            7 | 2007-02-05     | 9999-12-31  |
|        8 |            8 | 2007-02-05     | 9999-12-31  |
|        9 |            9 | 2007-02-05     | 9999-12-31  |
|       10 |           10 | 2007-02-05     | 9999-12-31  |
+----------+--------------+----------------+-------------+


In [ ]:
!mysql dw -e 'update date_dim set effective_date ="2007-02-05"'

In [ ]:
!mysql dw -e 'select * from date_dim'

+---------+------------+------------+-------+---------+------+----------------+-------------+
| date_sk | date       | month_name | month | quarter | year | effective_date | expiry_date |
+---------+------------+------------+-------+---------+------+----------------+-------------+
|       1 | 2005-10-31 | October    |    10 |       4 | 2005 | 2007-02-05     | 9999-12-31  |
+---------+------------+------------+-------+---------+------+----------------+-------------+


In [ ]:
!mysql dw -e 'select * from sales_order_fact'

+----------+-------------+------------+---------------+--------------+
| order_sk | customer_sk | product_sk | order_date_sk | order_amount |
+----------+-------------+------------+---------------+--------------+
|        1 |           1 |          2 |             1 |      1000.00 |
|        2 |           2 |          3 |             1 |      1000.00 |
|        3 |           3 |          4 |             1 |      4000.00 |
|        4 |           4 |          2 |             1 |      4000.00 |
|        5 |           5 |          3 |             1 |      6000.00 |
|        6 |           1 |          4 |             1 |      6000.00 |
|        7 |           2 |          2 |             1 |      8000.00 |
|        8 |           3 |          3 |             1 |      8000.00 |
|        9 |           4 |          4 |             1 |     10000.00 |
|       10 |           5 |          2 |             1 |     10000.00 |
+----------+-------------+------------+---------------+--------------+


#### Aggregate Queries
SUMMING ORDER AMOUNT FROM SALES_ORDER_FACT ACROSS ALL DIMENSIONS

In [ ]:
!mysql -udwid -ppw <scripts/across_all_dimensions.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
sum_of_order_amount
58000.00


QUERYING AGAINST DATE PRODUCT AND ORDER DIMENSION

In [ ]:

!mysql -udwid -ppw <scripts/across_date_product.sql 

mysql: [Warning] Using a password on the command line interface can be insecure.
customer_number	sum_of_order_amount
1	7000.00
2	9000.00
3	12000.00
4	14000.00
5	16000.00


In [ ]:
!mysql dw -e 'SELECT customer_number, SUM(order_amount) sum_of_order_amount \
FROM sales_order_fact a , customer_dim b WHERE a.customer_sk = b.customer_sk GROUP BY customer_number'

+-----------------+---------------------+
| customer_number | sum_of_order_amount |
+-----------------+---------------------+
|               1 |             7000.00 |
|               2 |             9000.00 |
|               3 |            12000.00 |
|               4 |            14000.00 |
|               5 |            16000.00 |
+-----------------+---------------------+


QUERYING AGAINST DATE CUSTOMER AND ORDER DIMENSION

In [ ]:
!mysql dw -e 'SELECT product_code, SUM(order_amount) sum_of_order_amount \
FROM sales_order_fact a , product_dim b WHERE a.product_sk = b.product_sk GROUP BY product_code'

+--------------+---------------------+
| product_code | sum_of_order_amount |
+--------------+---------------------+
|            1 |            15000.00 |
|            2 |            23000.00 |
|            3 |            20000.00 |
+--------------+---------------------+


QUERYING AGAINST DATE AND ORDER DIMENSION

In [ ]:
!mysql dw -e'SELECT customer_number, product_code, SUM(order_amount) sum_of_order_amount \
FROM sales_order_fact a , customer_dim b , product_dim c \
WHERE a.customer_sk = b.customer_sk AND a.product_sk = c.product_sk GROUP BY customer_number , product_code'

+-----------------+--------------+---------------------+
| customer_number | product_code | sum_of_order_amount |
+-----------------+--------------+---------------------+
|               1 |            2 |             1000.00 |
|               1 |            3 |             6000.00 |
|               2 |            1 |             1000.00 |
|               2 |            2 |             8000.00 |
|               3 |            1 |             8000.00 |
|               3 |            3 |             4000.00 |
|               4 |            2 |             4000.00 |
|               4 |            3 |            10000.00 |
|               5 |            1 |             6000.00 |
|               5 |            2 |            10000.00 |
+-----------------+--------------+---------------------+


## Chapter 4: Dimensional Queries
A dimensional query is a query in a dimensional data warehouse that joins the fact table and the dimension tables on one or more surrogate keys. 


**Applying Dimensional Queries**

In [ ]:
!mysql -udwid -ppw <scripts/dimensional_query_data.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e 'select * from order_dim'

+----------+--------------+----------------+-------------+
| order_sk | order_number | effective_date | expiry_date |
+----------+--------------+----------------+-------------+
|        1 |            1 | 2007-02-05     | 9999-12-31  |
|        2 |            2 | 2007-02-05     | 9999-12-31  |
|        3 |            3 | 2007-02-05     | 9999-12-31  |
|        4 |            4 | 2007-02-05     | 9999-12-31  |
|        5 |            5 | 2007-02-05     | 9999-12-31  |
|        6 |            6 | 2007-02-05     | 9999-12-31  |
|        7 |            7 | 2007-02-05     | 9999-12-31  |
|        8 |            8 | 2007-02-05     | 9999-12-31  |
|        9 |            9 | 2007-02-05     | 9999-12-31  |
|       10 |           10 | 2007-02-05     | 9999-12-31  |
|       11 |           11 | 2020-10-17     | 9999-12-31  |
|       12 |           12 | 2020-10-17     | 9999-12-31  |
|       13 |           13 | 2020-10-17     | 9999-12-31  |
|       14 |           14 | 2020-10-17     | 9999-12-31 

In [ ]:
# changing the effective date to "2007-02-06" for newly added orders
!mysql dw -e 'update order_dim set effective_date ="2007-02-06" where order_sk in (11,12,13,14,15,16) '

In [ ]:
!mysql dw -e 'select * from order_dim'

+----------+--------------+----------------+-------------+
| order_sk | order_number | effective_date | expiry_date |
+----------+--------------+----------------+-------------+
|        1 |            1 | 2007-02-05     | 9999-12-31  |
|        2 |            2 | 2007-02-05     | 9999-12-31  |
|        3 |            3 | 2007-02-05     | 9999-12-31  |
|        4 |            4 | 2007-02-05     | 9999-12-31  |
|        5 |            5 | 2007-02-05     | 9999-12-31  |
|        6 |            6 | 2007-02-05     | 9999-12-31  |
|        7 |            7 | 2007-02-05     | 9999-12-31  |
|        8 |            8 | 2007-02-05     | 9999-12-31  |
|        9 |            9 | 2007-02-05     | 9999-12-31  |
|       10 |           10 | 2007-02-05     | 9999-12-31  |
|       11 |           11 | 2007-02-06     | 9999-12-31  |
|       12 |           12 | 2007-02-06     | 9999-12-31  |
|       13 |           13 | 2007-02-06     | 9999-12-31  |
|       14 |           14 | 2007-02-06     | 9999-12-31 

In [ ]:
!mysql dw -e 'select * from date_dim'

+---------+------------+------------+-------+---------+------+----------------+-------------+
| date_sk | date       | month_name | month | quarter | year | effective_date | expiry_date |
+---------+------------+------------+-------+---------+------+----------------+-------------+
|       1 | 2005-10-31 | October    |    10 |       4 | 2005 | 2007-02-05     | 9999-12-31  |
|       2 | 2005-11-01 | November   |    11 |       4 | 2005 | 2020-10-17     | 9999-12-31  |
+---------+------------+------------+-------+---------+------+----------------+-------------+


In [ ]:
!mysql dw -e 'update date_dim set effective_date ="2007-02-05"'

In [ ]:
!mysql dw -e 'select * from date_dim'

+---------+------------+------------+-------+---------+------+----------------+-------------+
| date_sk | date       | month_name | month | quarter | year | effective_date | expiry_date |
+---------+------------+------------+-------+---------+------+----------------+-------------+
|       1 | 2005-10-31 | October    |    10 |       4 | 2005 | 2007-02-05     | 9999-12-31  |
|       2 | 2005-11-01 | November   |    11 |       4 | 2005 | 2007-02-05     | 9999-12-31  |
+---------+------------+------------+-------+---------+------+----------------+-------------+


In [ ]:
!mysql dw -e 'select * from sales_order_fact'

+----------+-------------+------------+---------------+--------------+
| order_sk | customer_sk | product_sk | order_date_sk | order_amount |
+----------+-------------+------------+---------------+--------------+
|        1 |           1 |          2 |             1 |      1000.00 |
|        2 |           2 |          3 |             1 |      1000.00 |
|        3 |           3 |          4 |             1 |      4000.00 |
|        4 |           4 |          2 |             1 |      4000.00 |
|        5 |           5 |          3 |             1 |      6000.00 |
|        6 |           1 |          4 |             1 |      6000.00 |
|        7 |           2 |          2 |             1 |      8000.00 |
|        8 |           3 |          3 |             1 |      8000.00 |
|        9 |           4 |          4 |             1 |     10000.00 |
|       10 |           5 |          2 |             1 |     10000.00 |
|       11 |           1 |          2 |             2 |     20000.00 |
|     

### Aggregate Queries
#### Daily Sales Aggregation


In [ ]:
!mysql dw  <scripts/daily_aggregation.sql

date	SUM(order_amount)	COUNT(*)
2005-10-31	58000.00	10
2005-11-01	195000.00	6


#### Annual Aggregation

In [ ]:
!mysql dw  <scripts/annual_aggregation.sql

year	product_name	customer_city	SUM(order_amount)	COUNT(*)
2005	Floppy Drive	Mechanicsburg	70000.00	5
2005	Floppy Drive	Pittsburgh	8000.00	1
2005	Hard Disk Drive	Mechanicsburg	46000.00	2
2005	Hard Disk Drive	Pittsburgh	34000.00	3
2005	LCD Panel	Mechanicsburg	61000.00	3
2005	LCD Panel	Pittsburgh	34000.00	2


### Specific Queries

A specific query selects and aggregates the facts on a specific dimension value. 

#### Monthly Storage Product Sales


In [ ]:
!mysql dw  <scripts/monthly_storage.sql 

product_name	month_name	year	SUM(order_amount)	COUNT(*)
Floppy Drive	November	2005	55000.00	2
Hard Disk Drive	November	2005	65000.00	2
Hard Disk Drive	October	2005	15000.00	3
Floppy Drive	October	2005	23000.00	4


#### Quarterly Sales in Mechanisburg

In [ ]:
!mysql dw  <scripts/quarterly_mechanicsburg.sql  

customer_city	quarter	year	SUM(order_amount)	COUNT(order_sk)
Mechanicsburg	4	2005	177000.00	10


###Inside-Out Queries
### Product Performer
This query gives the sales orders of products that have a monthly sales amount of 75,000 or more

In [ ]:
#Find sales orders of products that have a monthly sales amount of 75,000 or more
!mysql dw <scripts/monthly_product_performer.sql

ERROR 1055 (42000) at line 9: Expression #1 of ORDER BY clause is not in GROUP BY clause and contains nonaggregated column 'dw.c.month' which is not functionally dependent on columns in GROUP BY clause; this is incompatible with sql_mode=only_full_group_by


In [ ]:
!mysql dw -e 'SELECT month_name, year, product_name,\
 SUM(order_amount), COUNT(*) FROM sales_order_fact a \
  , product_dim b , date_dim c WHERE a.product_sk = b.product_sk \
  AND a.order_date_sk = c.date_sk GROUP BY month_name , year , product_name HAVING SUM(order_amount) >= 75000 ORDER BY month_name , year , product_name'

+------------+------+--------------+-------------------+----------+
| month_name | year | product_name | SUM(order_amount) | COUNT(*) |
+------------+------+--------------+-------------------+----------+
| November   | 2005 | LCD Panel    |          75000.00 |        2 |
+------------+------+--------------+-------------------+----------+


### Loyal Customer
It gives the customer(s) that placed more than five orders annually in the past eighteen months

In [ ]:
!mysql dw <scripts/loyal_customer.sql

customer_number	year	COUNT(*)
1	2005	4
